In [1]:
import pandas as pd
import numpy as np 
import warnings 
warnings.filterwarnings("ignore")

In [2]:
#1916-2016
movie_metadata=pd.read_csv("movie_metadata.csv")
data_movie=movie_metadata.copy()

data_movie=data_movie[["movie_title","genres","title_year","director_name","actor_1_name",
                       "actor_2_name","actor_3_name"]]

data_movie["genres"]=data_movie["genres"].str.replace("|"," ")


data_movie["movie_title"]=data_movie["movie_title"].str.lower()
data_movie["movie_title"]=data_movie["movie_title"].str.replace("part","")
data_movie["movie_title"]=data_movie["movie_title"].str.replace(":"," ")
data_movie["movie_title"]=data_movie["movie_title"].str.replace("-"," ")
data_movie["movie_title"]=data_movie["movie_title"].str.replace("  "," ")
data_movie["movie_title"]=data_movie["movie_title"].str.replace("   "," ")

data_movie["movie_title"]=data_movie["movie_title"].str.upper()
import roman 
movie_names=data_movie["movie_title"]
new_movies = []
for movie in movie_names:
    word_list = movie.split()
    new_word_list = []
    for word in word_list:
        try:
            number = roman.fromRoman(word)
            new_word_list.append(str(number))
        except roman.InvalidRomanNumeralError:
            new_word_list.append(word)
    new_movie = " ".join(new_word_list)
    new_movies.append(new_movie)

data_movie["movie_title"]=pd.DataFrame(new_movies)
data_movie["movie_title"]=data_movie["movie_title"].str.lower()
movie_1916_2016=data_movie.copy()

In [3]:
movie_1916_2016.sample(5)

,movie_title,genres,title_year,director_name,actor_1_name,actor_2_name,actor_3_name
3413,the singing detective,Comedy Crime Musical Mystery,2003.0,Keith Gordon,Robert Downey Jr.,Robin Wright,Alfre Woodard
1448,the pianist,Biography Drama War,2002.0,Roman Polanski,Emilia Fox,Frank Finlay,Ed Stoppard
3647,deadfall,Crime Drama Thriller,2012.0,Stefan Ruzowitzky,Charlie Hunnam,Olivia Wilde,Sissy Spacek
0,avatar,Action Adventure Fantasy Sci-Fi,2009.0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi
520,the league of extraordinary gentlemen,Action Adventure Fantasy Sci-Fi,2003.0,Stephen Norrington,Jason Flemyng,Max Ryan,Tony Curran


In [4]:
#2017-2022
def get_movies(year):
    link="https://en.wikipedia.org/wiki/List_of_American_films_of_"
    link=link+str(year)
    import pandas as pd
    df1 = pd.read_html(link, header=0)[2]
    df2 = pd.read_html(link, header=0)[3]
    df3 = pd.read_html(link, header=0)[4]
    df4 = pd.read_html(link, header=0)[5]
    df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
    return df

movie_2017=get_movies(2017)
movie_2018=get_movies(2018)
movie_2019=get_movies(2019)
movie_2020=get_movies(2020)
movie_2021=get_movies(2021)
movie_2022=get_movies(2022)

def get_columns(data):
    data=data[["Title","Cast and crew"]]
    return data

movie_2017=get_columns(movie_2017)
movie_2018=get_columns(movie_2018)
movie_2019=get_columns(movie_2019)
movie_2020=get_columns(movie_2020)
movie_2021=get_columns(movie_2021)
movie_2022=get_columns(movie_2022)

In [5]:
#2017 - 2019 
def get_genre_1(x):
    from tmdbv3api import TMDb,Movie
    import requests

    tmdb_movie = Movie()
    tmdb = TMDb()
    
    tmdb.api_key = ''
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

#2020-2022:

def get_genre_2(x):
    from tmdbv3api import TMDb,Movie
    import requests
    
    tmdb = TMDb() 
    tmdb_movie = Movie() 
    tmdb.api_key = ''
    genres = []
    result = tmdb_movie.search(x)
    if not result:
          return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                  genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN

            
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]


def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])


def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
 
  
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [6]:
#2017
movie_2017['genres'] = movie_2017['Title'].map(lambda x: get_genre_1(str(x)))
movie_2017["director_name"]= movie_2017['Cast and crew'].map(lambda x: get_director(x))
movie_2017["actor_name1"]= movie_2017['Cast and crew'].map(lambda x: get_actor1(x))
movie_2017["actor_name2"]= movie_2017['Cast and crew'].map(lambda x: get_actor2(x)) 
movie_2017["actor_name3"]= movie_2017['Cast and crew'].map(lambda x: get_actor3(x))

In [7]:
movie_2017.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
18,Rings,"F. Javier Gutiérrez (director); David Loucka, ...",Horror,F. Javier Gutiérrez,Matilda Lutz,Alex Roe,Johnny Galecki
231,Ferdinand,"Carlos Saldanha (director); Robert L. Baird, B...",Animation Family Adventure Comedy,Carlos Saldanha,John Cena,Kate McKinnon,Bobby Cannavale
186,Happy Death Day,Christopher B. Landon (director/screenplay); S...,Horror Mystery Comedy,Christopher B. Landon,Jessica Rothe,Israel Broussard,Ruby Modine
117,The Beguiled,Sofia Coppola (director/screenplay); Colin Far...,Drama,Sofia Coppola,Colin Farrell,Nicole Kidman,Kirsten Dunst
214,Mayhem,Joe Lynch (director); Matias Caruso (screenpla...,Action Comedy Horror,Joe Lynch,Steven Yeun,Samara Weaving,Steven Brand


In [8]:
#2018
movie_2018['genres'] = movie_2018['Title'].map(lambda x: get_genre_1(str(x)))
movie_2018["director_name"]= movie_2018['Cast and crew'].map(lambda x: get_director(x))
movie_2018["actor_name1"]= movie_2018['Cast and crew'].map(lambda x: get_actor1(x))
movie_2018["actor_name2"]= movie_2018['Cast and crew'].map(lambda x: get_actor2(x)) 
movie_2018["actor_name3"]= movie_2018['Cast and crew'].map(lambda x: get_actor3(x))


In [9]:
movie_2018.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
103,Terminal,Vaughn Stein (director/screenplay); Margot Rob...,Crime Drama Thriller,Vaughn Stein,Margot Robbie,Simon Pegg,Dexter Fletcher
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Western Crime Thriller Drama,Warwick Thornton,Bryan Brown,Sam Neill,NaN
49,Thoroughbreds,Cory Finley (director/screenplay); Olivia Cook...,Drama Thriller,Cory Finley,Olivia Cooke,Anya Taylor-Joy,Anton Yelchin
207,Hell Fest,"Gregory Plotkin (director); William Penick, Ch...",Horror,Gregory Plotkin,Bex Taylor-Klaus,Amy Forsyth,Reign Edwards
120,Ocean's 8,Gary Ross (director/screenplay); Olivia Milch ...,Crime Comedy,Gary Ross,Sandra Bullock,Cate Blanchett,Anne Hathaway


In [10]:
#2019
movie_2019['genres'] = movie_2019['Title'].map(lambda x: get_genre_1(str(x)))
movie_2019["director_name"]= movie_2019['Cast and crew'].map(lambda x: get_director(x))
movie_2019["actor_name1"]= movie_2019['Cast and crew'].map(lambda x: get_actor1(x))
movie_2019["actor_name2"]= movie_2019['Cast and crew'].map(lambda x: get_actor2(x)) 
movie_2019["actor_name3"]= movie_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [11]:
movie_2019.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
47,The Beach Bum,Harmony Korine (director/screenplay); Matthew ...,Comedy,Harmony Korine,Matthew McConaughey,Zac Efron,Jonah Hill
93,The Perfection,Richard Shepard (director/screenplay); Eric Ch...,Horror Thriller Drama,Richard Shepard,Allison Williams,Logan Browning,Steven Weber
24,Happy Death Day 2U,Christopher Landon (director/screenplay); Jess...,Comedy Horror Science Fiction,Christopher Landon,Jessica Rothe,Israel Broussard,Phi Vu
73,Buffaloed,Tanya Wexler (director); Brian Sacca (screenpl...,Comedy Drama,Tanya Wexler,Zoey Deutch,Jermaine Fowler,Judy Greer
163,Ad Astra,James Gray (director/screenplay); Ethan Gross ...,Science Fiction Drama,James Gray,Brad Pitt,Tommy Lee Jones,Ruth Negga


In [12]:
#2020
movie_2020['genres'] = movie_2020['Title'].map(lambda x: get_genre_2(str(x)))
movie_2020["director_name"]= movie_2020['Cast and crew'].map(lambda x: get_director(str(x)))
movie_2020["actor_name1"]= movie_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))
movie_2020["actor_name2"]= movie_2020['Cast and crew'].map(lambda x: get_actor2(str(x))) 
movie_2020["actor_name3"]= movie_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [13]:
movie_2020.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
153,The Babysitter: Killer Queen,McG (director/screenplay); Dan Lagana (screenp...,Comedy Horror,McG,Judah Lewis,Emily Alyn Lind,Jenna Ortega
226,Fatman,"Eshom Nelms, Ian Nelms (directors/screenplay);...",Action Comedy Fantasy,"Eshom Nelms, Ian Nelms (directors/screenplay);...",Mel Gibson,Walton Goggins,Marianne Jean-Baptiste
158,Rent-A-Pal,Jon Stevenson (director/screenplay); Wil Wheat...,Horror Thriller,Jon Stevenson,Wil Wheaton,Brian Landis Folkins,Kathleen Brady
71,All Day and a Night,Joe Robert Cole (director/screenplay); Ashton ...,Drama,Joe Robert Cole,Ashton Sanders,Jeffrey Wright,Yahya Abdul-Mateen II
269,Wonder Woman 1984,Patty Jenkins (director/screenplay); Geoff Joh...,Action Adventure Fantasy,Patty Jenkins,Gal Gadot,Chris Pine,Kristen Wiig


In [14]:
#2021
movie_2021['genres'] = movie_2021['Title'].map(lambda x: get_genre_2(str(x)))
movie_2021["director_name"]= movie_2021['Cast and crew'].map(lambda x: get_director(str(x)))
movie_2021["actor_name1"]= movie_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))
movie_2021["actor_name2"]= movie_2021['Cast and crew'].map(lambda x: get_actor2(str(x))) 
movie_2021["actor_name3"]= movie_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [15]:
movie_2021.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
48,The SpongeBob Movie: Sponge on the Run,"Tim Hill (director/screenplay); Tom Kenny, Awk...",Family Animation Fantasy Adventure Comedy,Tim Hill,Tom Kenny,Awkwafina,Matt Berry
297,A Mouthful of Air,Amy Koppelman (director/screenplay); Amanda Se...,Drama,Amy Koppelman,Amanda Seyfried,Finn Wittrock,Jennifer Carpenter
233,Queenpins,"Aron Gaudet, Gita Pullapilly (directors/screen...",Comedy Crime,"Aron Gaudet, Gita Pullapilly (directors/screen...",Kristen Bell,Kirby Howell-Baptiste,Paul Walter Hauser
206,Reminiscence,"Lisa Joy (director/screenplay); Hugh Jackman, ...",Mystery Science Fiction,Lisa Joy,Hugh Jackman,Rebecca Ferguson,Thandiwe Newton
360,NaN,NaN,Drama,nan,nan,NaN,NaN


In [16]:
#2022
movie_2022['genres'] = movie_2022['Title'].map(lambda x: get_genre_2(str(x)))
movie_2022["director_name"]= movie_2022['Cast and crew'].map(lambda x: get_director(str(x)))
movie_2022["actor_name1"]= movie_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))
movie_2022["actor_name2"]= movie_2022['Cast and crew'].map(lambda x: get_actor2(str(x))) 
movie_2022["actor_name3"]= movie_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [17]:
movie_2022.sample(5)

,Title,Cast and crew,genres,director_name,actor_name1,actor_name2,actor_name3
133,Spiderhead,"Joseph Kosinski (director); Rhett Reese, Paul ...",Science Fiction Thriller,Joseph Kosinski,Chris Hemsworth,Miles Teller,Jurnee Smollett
115,Top Gun: Maverick,"Joseph Kosinski (director); Ehren Kruger, Eric...",Action Drama,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly
188,Emily the Criminal,John Patton Ford (director/screenplay); Aubrey...,Crime Drama Thriller,John Patton Ford,Aubrey Plaza,Theo Rossi,Megalyn Echikunwoke
203,One Way,Andrew Baird (director); Ben Conway (screenpla...,Action Crime Thriller,Andrew Baird,Colson Baker,Kevin Bacon,Travis Fimmel
87,The Bad Guys,Pierre Perifel (director); Etan Cohen (screenp...,Animation Action Adventure Crime Comedy Family,Pierre Perifel,Sam Rockwell,Marc Maron,Craig Robinson


In [18]:
#delete colums 
movie_2017=movie_2017.drop("Cast and crew",axis=1)
movie_2018=movie_2018.drop("Cast and crew",axis=1)
movie_2019=movie_2019.drop("Cast and crew",axis=1)
movie_2020=movie_2020.drop("Cast and crew",axis=1)
movie_2021=movie_2021.drop("Cast and crew",axis=1)
movie_2022=movie_2022.drop("Cast and crew",axis=1)

In [19]:
#new columns :
movie_1916_2016.rename(columns={"movie_title":"Title",
                               "genres":"genres",
                               "director_name":"director_name",
                               "actor_1_name":"actor_name1",
                               "actor_2_name":"actor_name2",
                               "actor_3_name":"actor_name3"},inplace=True)


movie_1916_2016["genres_director_actor"]=movie_1916_2016['actor_name1'] + ' ' + movie_1916_2016['actor_name2'] + ' '+ movie_1916_2016['actor_name3'] + ' '+ movie_1916_2016['director_name'] +' ' + movie_1916_2016['genres']
movie_2017["genres_director_actor"]=movie_2017['actor_name1'] + ' ' + movie_2017['actor_name2'] + ' '+ movie_2017['actor_name3'] + ' '+ movie_2017['director_name'] +' ' + movie_2017['genres']
movie_2018["genres_director_actor"]=movie_2018['actor_name1'] + ' ' + movie_2018['actor_name2'] + ' '+ movie_2018['actor_name3'] + ' '+ movie_2018['director_name'] +' ' + movie_2018['genres']
movie_2019["genres_director_actor"]=movie_2019['actor_name1'] + ' ' + movie_2019['actor_name2'] + ' '+ movie_2019['actor_name3'] + ' '+ movie_2019['director_name'] +' ' + movie_2019['genres']
movie_2020["genres_director_actor"]=movie_2020['actor_name1'] + ' ' + movie_2020['actor_name2'] + ' '+ movie_2020['actor_name3'] + ' '+ movie_2020['director_name'] +' ' + movie_2020['genres']
movie_2021["genres_director_actor"]=movie_2021['actor_name1'] + ' ' + movie_2021['actor_name2'] + ' '+ movie_2021['actor_name3'] + ' '+ movie_2021['director_name'] +' ' + movie_2021['genres']
movie_2022["genres_director_actor"]=movie_2022['actor_name1'] + ' ' + movie_2022['actor_name2'] + ' '+ movie_2022['actor_name3'] + ' '+ movie_2022['director_name'] +' ' + movie_2022['genres']

In [20]:
movie_2017["title_year"]=2017.0
movie_2018["title_year"]=2018.0
movie_2019["title_year"]=2019.0
movie_2020["title_year"]=2020.0
movie_2021["title_year"]=2021.0
movie_2022["title_year"]=2022.0

In [21]:
#concat
all_movies = pd.concat([movie_1916_2016, movie_2017, movie_2018, movie_2019, movie_2020, movie_2021, movie_2022])

In [22]:
all_movies.sample(5)

,Title,genres,title_year,director_name,actor_name1,actor_name2,actor_name3,genres_director_actor
142,Phineas and Ferb the Movie: Candace Against th...,Family Animation Science Fiction Comedy TV Movie,2020.0,Bob Bowen,Vincent Martella,Ashley Tisdale,David Errigo Jr.,Vincent Martella Ashley Tisdale David Errigo J...
2254,pooh's heffalump movie,Animation Family Fantasy Mystery,2005.0,Frank Nissen,Kath Soucie,Brenda Blethyn,John Fiedler,Kath Soucie Brenda Blethyn John Fiedler Frank ...
193,noah,Action Adventure Drama,2014.0,Darren Aronofsky,Anthony Hopkins,Emma Watson,Logan Lerman,Anthony Hopkins Emma Watson Logan Lerman Darre...
2023,think like a man too,Comedy Romance,2014.0,Tim Story,Romany Malco,Regina Hall,David Walton,Romany Malco Regina Hall David Walton Tim Stor...
4421,snow angels,Drama Romance,2007.0,David Gordon Green,Michael Angarano,Connor Paolo,Tom Noonan,Michael Angarano Connor Paolo Tom Noonan David...


In [23]:
all_movies.shape

(6686, 8)

In [24]:
all_movies=all_movies.dropna()

In [25]:
all_movies.shape

(6415, 8)

In [26]:
all_movies.drop_duplicates( inplace=True)

In [27]:
all_movies.shape

(6293, 8)

In [28]:
#saving data :
movie_1916_2016.to_csv("movie_1916_2016.csv")
movie_2017.to_csv("movie_2017.csv")
movie_2018.to_csv("movie_2018.csv")
movie_2019.to_csv("movie_2019.csv")
movie_2020.to_csv("movie_2020.csv")
movie_2021.to_csv("movie_2021.csv")
movie_2022.to_csv("movie_2022.csv")



all_movies.to_csv("all_movies.csv")